In [14]:
#r "nuget: FSharp.Text.RegexProvider"

open System
open FSharp.Text
open System.Collections.Concurrent

type Move = { Count : int; From : int; To : int }
type MoveRegex = FSharp.Text.RegexProvider.Regex< @"^move (?<Count>\d+) from (?<From>\d+) to (?<To>\d+)$" >
type BoxRegex = FSharp.Text.RegexProvider.Regex< @"^\[(?<Contents>[A-Z])\]?" >
let moveRegex = MoveRegex()
let boxRegex = BoxRegex()

let parseText (lines : string[]) =
    let header =
        lines 
        |> Seq.takeWhile (not << String.IsNullOrEmpty)
        |> List.ofSeq
        |> List.rev

    let moves = 
        lines
        |> Array.skip (header.Length + 1)
        |> Array.map (
            fun l -> 
                let m = moveRegex.TypedMatch l
                { Count = int m.Count.Value; From = int m.From.Value; To = int m.To.Value }
            )

    let stackNames = 
        let labels = List.head header
        labels.Split([|' '|], StringSplitOptions.RemoveEmptyEntries)
        |> Array.map int

    let stacks =
        stackNames
        |> Array.map (fun name -> (name, ConcurrentStack()))
        |> Map.ofArray

    let boxes = 
        header
        |> List.skip 1
        |> Seq.collect (
            fun row ->
                (row :> char seq)
                |> Seq.chunkBySize 4 
                |> Seq.map String
                |> Seq.zip stackNames
                |> Seq.choose (fun (n, t) -> boxRegex.TryTypedMatch t |> Option.map (fun r -> n, r.Contents.Value[0]))
            )
    
    for (stackLabel, contents) in boxes do
        stacks[stackLabel].Push contents

    stacks, moves


Installed Packages FSharp.Text.RegexProvider, 2.1.0

In [17]:
#r "nuget:FsUnit"

open FsUnitTyped

let lines = 
    [|
        "    [D]    "
        "[N] [C]    "
        "[Z] [M] [P]"
        " 1   2   3 "
        ""
        "move 1 from 2 to 1"
        "move 2 from 2 to 1"
        "move 1 from 1 to 2"
    |]

let (stacks, moves) = parseText lines

moves |> shouldEqual [| { Count = 1; From = 2; To = 1 }; { Count = 2; From = 2; To = 1}; { Count = 1; From = 1; To = 2} |]
stacks[1] |> List.ofSeq |> shouldEqual [ 'N'; 'Z' ]
stacks[2] |> List.ofSeq |> shouldEqual [ 'D'; 'C'; 'M' ]
stacks[3] |> List.ofSeq |> shouldEqual [ 'P' ]

Installed Packages FsUnit, 5.1.0

In [21]:
let applyMove (stacks : Map<_, ConcurrentStack<_>>) (move : Move) =
    for _ = 1 to move.Count do
        match stacks[move.From].TryPop() with
        | true, box -> stacks[move.To].Push box
        | false, _ -> failwith "Tried to move from empty stack"

let tops (stacks : Map<_, ConcurrentStack<_>>) =
    stacks 
    |> Map.map (fun _ s -> Seq.head s)
    |> Map.values
    |> Array.ofSeq
    |> String

In [24]:
open System.IO

let sourcePath = Path.Combine(__SOURCE_DIRECTORY__, "input_05.txt")

let (stacks, moves) = 
    File.ReadAllLines(sourcePath) 
    |> parseText

for move in moves do
    applyMove stacks move

let result = 
    stacks 
    |> tops

Stopped due to error


Error: input.fsx (10,15)-(10,21) typecheck error This expression was expected to have type
    'Map<int,ConcurrentStack<obj>>'    
but here has type
    'Move[]'    
input.fsx (10,22)-(10,26) typecheck error This expression was expected to have type
    'Move'    
but here has type
    'Collections.Generic.KeyValuePair<int,ConcurrentStack<char>>'    
input.fsx (14,8)-(14,12) typecheck error Type mismatch. Expecting a
    'Move[] -> obj'    
but given a
    'Map<'a,ConcurrentStack<char>> -> String'    
The type 'Move[]' does not match the type 'Map<'a,ConcurrentStack<char>>'

In [23]:
printfn "Top boxes are %s" result

Top boxes are CMP
